# Login to weights & biases for experiment tracking

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip install datasets
!pip install peft
!pip install bitsandbytes
!pip install trl
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 7.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 84.6 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1


In [3]:
import json
import os

import transformers
import torch

import accelerate
from transformers import BitsAndBytesConfig
import bitsandbytes
from accelerate import Accelerator

import pandas as pd
from pathlib import Path
from datasets import load_dataset
from peft import PeftModel, get_peft_model,LoraConfig,TaskType,prepare_model_for_kbit_training
from torch import cuda, bfloat16
from contextlib import nullcontext
from tqdm import tqdm
from itertools import chain
from torch.utils.data import Dataset
import datasets

from trl import SFTTrainer, setup_chat_format, SFTConfig

from transformers import TrainingArguments
from transformers import Trainer
from transformers import LlamaForCausalLM, LlamaConfig
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoModelForSeq2SeqLM
import wandb

In [4]:
wandb.login()
epoch = 2
lr = 0.002
bs = 1
quantized = 'bfloat16'
max_length = 512
#  2. Capture a dictionary of hyperparameters

run = wandb.init(
    # Set the project where this run will be logged
    project="testing",
    config = {"quantized": quantized},
    job_type="train" 
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 

# load model

In [4]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoModelForSeq2SeqLM
import torch
import bitsandbytes


config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)


# Define the model name or checkpoint path
model_name = "meta-llama/Llama-3.2-1B"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model
model = AutoModelForCausalLM.from_pretrained(model_name,device_map='cuda', quantization_config=config)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/885 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

# dataset

In [6]:
from datasets import load_dataset
dataset = load_dataset("knkarthick/dialogsum")
print(dataset)
small_dataset = dataset['train'].shuffle(seed=42).select(range(100))
dataset["train"] = small_dataset

small_dataset = dataset['validation'].shuffle(seed=42).select(range(20))
dataset["validation"] = small_dataset

small_dataset = dataset.get('test').shuffle(seed=42).select(range(20))
dataset["test"] = small_dataset
print('--------------')
print(f'processed dataset: {dataset}')

README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})
--------------
processed dataset: DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 20
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 20
    })
})


In [7]:
EOS_TOKEN = tokenizer.eos_token
def format_template(row):
    dialogue = row["dialogue"].strip('\n')
    prompt = f'''
    Summarize the following conversation.
    
    Conversation:
    {dialogue}

    Summary: 
    {row["summary"]}
    '''

    row["text"] = prompt + EOS_TOKEN
    return row

dataset = dataset.map(
    format_template,
    num_proc=4,
)

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/20 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/20 [00:00<?, ? examples/s]

In [8]:
dataset['train']['text'][1]

"\n    Summarize the following conversation.\n    \n    Conversation:\n    #Person1#: Mister Ewing said we should show up at the conference center at 4:00 o'clock, right?\n#Person2#: Yes, he especially asked us not to be late. Some of the people from our east york branch office are coming, and he wants to make a good impression on them. How are you getting there?\n#Person1#: I was thinking of taking my car, but I think I'm just going to take the underground, because there is construction on the highway. What about you?\n#Person2#: I'll be taking the underground as well. Why don't we go together? I've been to the conference center only once, and I'm not sure if I can find my way around there.\n\n    Summary: \n    #Person1# and #Person2# plan to take the underground together to the conference center because Mr. Ewing asks them not to be late.\n    <|end_of_text|>"

# Initial Response

In [9]:
prompt = """
Summarize the following conversation.

Coversation:
#Person1#: Mister Ewing said we should show up at the conference center at 4:00 o'clock, right?
#Person2#: Yes, he especially asked us not to be late. Some of the people from our east york branch office are coming, and he wants to make a good impression on them. How are you getting there?
#Person1#: I was thinking of taking my car, but I think I'm just going to take the underground, because there is construction on the highway. What about you?
#Person2#: I'll be taking the underground as well. Why don't we go together? I've been to the conference center only once, and I'm not sure if I can find my way around there.

Summary:
"""
inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)
#print(f'INPUT PROMPT:\n{prompt}')
print(output)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Summarize the following conversation.

Coversation:
#Person1#: Mister Ewing said we should show up at the conference center at 4:00 o'clock, right?
#Person2#: Yes, he especially asked us not to be late. Some of the people from our east york branch office are coming, and he wants to make a good impression on them. How are you getting there?
#Person1#: I was thinking of taking my car, but I think I'm just going to take the underground, because there is construction on the highway. What about you?
#Person2#: I'll be taking the underground as well. Why don't we go together? I've been to the conference center only once, and I'm not sure if I can find my way around there.

Summary:
#Person1# and #Person2# are both attending a conference. #Person1# wants to show up at the conference center at 4:00 o'clock, #Person2# wants to show up at 4:00 o'clock, but #Person1# is worried that he will be late. #Person2# is worried that he will be late. #Person1# asks #Person2# if he can show up at the conf

# Training

In [10]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
)


training_arguments = SFTConfig(
    output_dir='./trained',
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="adamw_torch",
    num_train_epochs=20,
    eval_strategy="steps",
    eval_steps=10,
    logging_steps=5,
    warmup_steps=10,
    logging_strategy="steps",
    dataset_text_field="text",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    max_seq_length=512,
    report_to="wandb",
    save_strategy="no"
)


trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
10,1.810300,1.842821
20,1.701400,1.713877
30,1.797000,1.702748
40,1.674800,1.686877
50,1.589000,1.671448
60,1.348000,1.695241
70,1.346600,1.696154
80,1.525200,1.719518
90,1.491200,1.739847
100,1.305700,1.720833


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=1000, training_loss=0.29391394935548304, metrics={'train_runtime': 1272.1633, 'train_samples_per_second': 1.572, 'train_steps_per_second': 0.786, 'total_flos': 3142478157004800.0, 'train_loss': 0.29391394935548304, 'epoch': 20.0})

# Trained Response

In [12]:
prompt = """
Summarize the following conversation.

Conversation:
#Person1#: Mister Ewing said we should show up at the conference center at 4:00 o'clock, right?
#Person2#: Yes, he especially asked us not to be late. Some of the people from our east york branch office are coming, and he wants to make a good impression on them. How are you getting there?
#Person1#: I was thinking of taking my car, but I think I'm just going to take the underground, because there is construction on the highway. What about you?
#Person2#: I'll be taking the underground as well. Why don't we go together? I've been to the conference center only once, and I'm not sure if I can find my way around there.

Summary:
"""
inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)
print(f'INPUT PROMPT:\n{prompt}')
print(output)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT PROMPT:

Summarize the following conversation.

Conversation:
#Person1#: Mister Ewing said we should show up at the conference center at 4:00 o'clock, right?
#Person2#: Yes, he especially asked us not to be late. Some of the people from our east york branch office are coming, and he wants to make a good impression on them. How are you getting there?
#Person1#: I was thinking of taking my car, but I think I'm just going to take the underground, because there is construction on the highway. What about you?
#Person2#: I'll be taking the underground as well. Why don't we go together? I've been to the conference center only once, and I'm not sure if I can find my way around there.

Summary:


Summarize the following conversation.

Conversation:
#Person1#: Mister Ewing said we should show up at the conference center at 4:00 o'clock, right?
#Person2#: Yes, he especially asked us not to be late. Some of the people from our east york branch office are coming, and he wants to make a good i

alternatively, you could push to huggingface using: 

*Take note that the files you are pushing is only the adapter. You will need to merge it with the base model later on.

In [13]:
trainer.tokenizer.push_to_hub('ww-fine-tuned-adapter-v9',private=True)
trainer.model.push_to_hub('ww-fine-tuned-adapter-v9',private=True)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/WeiWen21/ww-fine-tuned-adapter-v9/commit/356609010c120dc9757de139a8c4dcdc27761a4a', commit_message='Upload model', commit_description='', oid='356609010c120dc9757de139a8c4dcdc27761a4a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/WeiWen21/ww-fine-tuned-adapter-v9', endpoint='https://huggingface.co', repo_type='model', repo_id='WeiWen21/ww-fine-tuned-adapter-v9'), pr_revision=None, pr_num=None)

# Evaluation

In [10]:
run = wandb.init(
    # Set the project where this run will be logged
    project="llama-fine-tune",
    job_type="evaluate" 
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [11]:
!pip install evaluate
!pip install bert_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00


In [ ]:
import evaluate
#evaluate.list_evaluation_modules()
bleu = evaluate.load('evaluate-metric/google_bleu')
bert = evaluate.load('evaluate-metric/bertscore')

In [13]:
predictions = []
gt = []
wandb_summary = []

for row in dataset['test']:
    
    prompt = f"""
    Summarize the following conversation.

    Conversation:
    {row['dialogue']}
    
    Summary:
    """
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=200,
        )[0], 
        skip_special_tokens=True
    )


    print(output.split("Summary:")[1].strip('/n'))
    current_pred = output.split("Summary:")[1]
    current_gt = row['summary']
    predictions.append(current_pred)
    gt.append(current_gt)
    bleu_score = bleu.compute(predictions=[current_pred],references=[current_gt])['google_bleu']
    bert_score = bert.compute(predictions=[current_pred],references=[current_gt],lang='en')
    precision, recall, f1 = bert_score['precision'][0], bert_score['recall'][0], bert_score['f1'][0]

    wandb.log(
        {'bleu_score':bleu_score,
        'precision':precision,
        'recall':recall,
        'f1':f1}
    )
    
    wandb_summary.append([prompt,current_gt,current_pred,bleu_score,precision,recall,f1])
    

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



     #Person1# and #Person2# are talking about haj, which is a pilgrimage to Mecca for muslims. #Person1# asks #Person2# what muslims do on the pilgrimage. #Person2# says that muslims must face towards Mecca and that the sacred black stone is around the large, sacred well. #Person1# asks #Person2# if there is something magical about the place. #Person2# says that people believe that going to Lourdes will cure them and that there is no magic in the place.
    


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...


     Person 1 asks Person 2 to take off his jacket and shirt. Then Person 2 holds up Person 1's right arm. Person 1 asks Person 2 if it hurts. Person 2 says yes, and Person 1 asks if Person 2 doesn't feel anything else. Person 2 says no, and Person 1 asks if Person 2 can stay in the hospital for the night.
     Person 1 thinks Person 2 doesn't feel anything else because Person 2 doesn't feel anything in Person 2's legs.
     Person 2 stays in the hospital for the night.
     Person 1 takes Person 2's shoulder X-ray in the morning.
     Person 1 and Person 2 decide to have Person 2 stay in the hospital for the next few days.
     Person 1 explains to the nurse that Person 2 doesn't feel anything else.
     Person 1 and Person 2 take Person 2 to the hospital every day.



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



     #Person1# likes football, basketball, swimming and golf. #Person2# likes football, basketball, swimming and tennis. #Person1# thinks that extreme sports are only for a small number of people. #Person2# thinks that there are many people who follow rugby in his country.
    


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



     #Person1# and #Person2# talk about #Person2#'s diet. #Person1# offers to get #Person2# a salad. #Person2# accepts. #Person1# then offers to get #Person2# some soup from the restaurant. #Person2# accepts. #Person1# then invites #Person2# to the party afterwards.
    

     #Person1# invites #Person2# to come into his office. #Person2# greets #Person1# and says, "Good morning. Thank you." #Person1# asks #Person2# if he has any writing experience and #Person2# says, "Yes, I have written for several top newspapers in the country. I'm also in the process of writing my first novel." #Person1# tells #Person2# why he wants him on staff.
    


In [14]:
output_df = pd.DataFrame(wandb_summary,columns=['Prompt', 'Ground Truth', 'Model Answer','BLEU_SCORE','Precision', 'Recall','f1'])
avg_BLEU = output_df['BLEU_SCORE'].mean()
avg_precision = output_df['Precision'].mean()
avg_recall = output_df['Recall'].mean()
avg_f1 = output_df['f1'].mean()
output_df

,Prompt,Ground Truth,Model Answer,BLEU_SCORE,Precision,Recall,f1
0,\n Summarize the following conversation.\n\...,#Person1# and #Person2# talk about pilgrims ar...,\n #Person1# and #Person2# are talking abo...,0.117949,0.869010,0.894844,0.881738
1,\n Summarize the following conversation.\n\...,#Person1# is sending a package with the help o...,\n #Person1# wants to send a package by fi...,0.075221,0.861336,0.913869,0.886825
2,\n Summarize the following conversation.\n\...,#Person1# invites Lily to picnic this weekend....,\n Lily will take part in the picnic at th...,0.035912,0.782955,0.875399,0.826600
3,\n Summarize the following conversation.\n\...,#Person1# makes drinks for #Person2# at a bar.,\n #Person1# can get #Person2# something t...,0.048951,0.845604,0.909002,0.876157
4,\n Summarize the following conversation.\n\...,Lin Fang and Lucy are talking about their favo...,\n Lin Fang and Lucy have different favori...,0.018519,0.742482,0.881741,0.806142
5,\n Summarize the following conversation.\n\...,#Person2# tells #Person1# that he falls in lov...,\n #Person1# is talking about a neighbor h...,0.112903,0.894168,0.890143,0.892151
6,\n Summarize the following conversation.\n\...,Mr. Smith asks #Person1# to help find his trai...,\n Person 1: How can I help you?\n Per...,0.024922,0.766385,0.850543,0.806274
7,\n Summarize the following conversation.\n\...,#Person1# congratulates #Person2# on #Person2#...,\n Person 1. You're going to set up your o...,0.016746,0.800838,0.805873,0.803347
8,\n Summarize the following conversation.\n\...,#Person1# gets a crank call and is angry about...,\n #Person1# is calling #Person2# who he k...,0.047619,0.842209,0.892153,0.866462
9,\n Summarize the following conversation.\n\...,#Person1# introduces the weekend driving cours...,"\n Person 1: Good afternoon, what can I do...",0.011662,0.769268,0.839741,0.802961


In [15]:
col2str = {x: 'string' for x in ['Prompt', 'Ground Truth', 'Model Answer']}
wandb.log({'avg_BLEU':avg_BLEU})
wandb.log({'avg_precision':avg_precision})
wandb.log({'avg_recall':avg_recall})
wandb.log({'avg_f1':avg_f1})
wandb.log({'results': wandb.Table(dataframe=output_df.astype(col2str))})

# Load and merge qlora weights using this [script](https://gist.github.com/ChrisHayduk/1a53463331f52dca205e55982baf9930)

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip install peft
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.1 MB/s eta 0:00:0000:0100:01


In [5]:
"""
The code below combines approaches published by both @eugene-yh and @jinyongyoo on Github. 
Thanks for the contributions guys!
"""

import torch
import peft
import json
import shutil
from peft.utils import _get_submodules
import os
import bitsandbytes as bnb
from bitsandbytes.functional import dequantize_4bit
from peft import PeftModel
from transformers import AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig, CodeLlamaTokenizer,AutoTokenizer
import gc
import copy

def save_model(model, tokenizer, to):
    print(f"Saving dequantized model to {to}...")
    model.save_pretrained(to)
    tokenizer.save_pretrained(to)
    config_data = json.loads(open(os.path.join(to, 'config.json'), 'r').read())
    config_data.pop("quantization_config", None)  # Remove quantization config
    config_data.pop("pretraining_tp", None)  # Remove any other TP settings
    with open(os.path.join(to, 'config.json'), 'w') as config:
        config.write(json.dumps(config_data, indent=2))
    
def dequantize_model(model, tokenizer, to='./dequantized_model', dtype=torch.bfloat16, device="cpu"):
    """
    'model': the peftmodel you loaded with qlora.
    'tokenizer': the model's corresponding hf's tokenizer.
    'to': directory to save the dequantized model
    'dtype': dtype that the model was trained using
    'device': device to load the model to
    """

    # Delete the model object if it exists
    if os.path.exists(to):
        shutil.rmtree(to)

    os.makedirs(to, exist_ok=True)

    cls = bnb.nn.Linear4bit

    with torch.no_grad():
        for name, module in model.named_modules():
            if isinstance(module, cls):
                print(f"Dequantizing `{name}`...")
                quant_state = copy.deepcopy(module.weight.quant_state)

                quant_state.dtype = dtype

                weights = dequantize_4bit(module.weight.data, quant_state=quant_state, quant_type="nf4").to(dtype)

                new_module = torch.nn.Linear(module.in_features, module.out_features, bias=None, dtype=dtype)
                new_module.weight = torch.nn.Parameter(weights)
                new_module.to(device=device, dtype=dtype)

                parent, target, target_name = _get_submodules(model, name)
                setattr(parent, target_name, new_module)

        model.is_loaded_in_4bit = False

        save_model(model, tokenizer, to)
        
        return model
        

model_path = "meta-llama/Llama-3.2-1B"
adapter_path = "WeiWen21/ww-fine-tuned-adapter-v9"

quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        )

try:
    print(f"Starting to load the model {model_path} into memory")

    model = LlamaForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        quantization_config=quantization_config,
        device_map="cuda"
    )
    print(model)
    
    tok = AutoTokenizer.from_pretrained(model_path)
    
    model.resize_token_embeddings(len(tok))
    
    # Note: This function outputs the dequantized model without merging the adapter yet
    # The code below it will merge the adapter and then save it to disk
    model = dequantize_model(model, tok, to='dequantized-model')
    model.eval()
    print(model)
    model = PeftModel.from_pretrained(model = model, model_id = adapter_path, ignore_mismatched_sizes=True)
    print(model)
    model = model.merge_and_unload()
    print(model)
    
    print(f"Successfully loaded the model {model_path} into memory")
    
    # Note that the output folder here should be different than the one you used for dequantize_model
    # This save will output the model merged with LoRA weights
    save_model(model, tok, "ww_merged")
    
    print(f"Successfully saved merged model {model_path} to disk")

except Exception as e:
    print(f"An error occurred: {e}")

    # Delete the model object if it exists
    if 'model' in locals():
        del model

    # Clear the GPU cache
    torch.cuda.empty_cache()

    # Run the garbage collection
    gc.collect()

    print("Model, GPU cache, and garbage have been cleared.")

Starting to load the model meta-llama/Llama-3.2-1B into memory
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
   

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Dequantizing `model.layers.0.self_attn.q_proj`...
Dequantizing `model.layers.0.self_attn.k_proj`...
Dequantizing `model.layers.0.self_attn.v_proj`...
Dequantizing `model.layers.0.self_attn.o_proj`...
Dequantizing `model.layers.0.mlp.gate_proj`...
Dequantizing `model.layers.0.mlp.up_proj`...
Dequantizing `model.layers.0.mlp.down_proj`...
Dequantizing `model.layers.1.self_attn.q_proj`...
Dequantizing `model.layers.1.self_attn.k_proj`...
Dequantizing `model.layers.1.self_attn.v_proj`...
Dequantizing `model.layers.1.self_attn.o_proj`...
Dequantizing `model.layers.1.mlp.gate_proj`...
Dequantizing `model.layers.1.mlp.up_proj`...
Dequantizing `model.layers.1.mlp.down_proj`...
Dequantizing `model.layers.2.self_attn.q_proj`...
Dequantizing `model.layers.2.self_attn.k_proj`...
Dequantizing `model.layers.2.self_attn.v_proj`...
Dequantizing `model.layers.2.self_attn.o_proj`...
Dequantizing `model.layers.2.mlp.gate_proj`...
Dequantizing `model.layers.2.mlp.up_proj`...
Dequantizing `model.layers.2.m

adapter_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Line

Push the merged model

In [6]:
tok.push_to_hub("WeiWen21/fine-tuned-merged-model-v6")
model.push_to_hub("WeiWen21/fine-tuned-merged-model-v6")

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/WeiWen21/fine-tuned-merged-model-v6/commit/c82b3c7cda3ee17ba45a60a3213fc70e4e099823', commit_message='Upload LlamaForCausalLM', commit_description='', oid='c82b3c7cda3ee17ba45a60a3213fc70e4e099823', pr_url=None, repo_url=RepoUrl('https://huggingface.co/WeiWen21/fine-tuned-merged-model-v6', endpoint='https://huggingface.co', repo_type='model', repo_id='WeiWen21/fine-tuned-merged-model-v6'), pr_revision=None, pr_num=None)

In [7]:
model.to('cuda')
prompt = """
Summarize the following conversation.

Conversation:
#Person1#: Mister Ewing said we should show up at the conference center at 4:00 o'clock, right?
#Person2#: Yes, he especially asked us not to be late. Some of the people from our east york branch office are coming, and he wants to make a good impression on them. How are you getting there?
#Person1#: I was thinking of taking my car, but I think I'm just going to take the underground, because there is construction on the highway. What about you?
#Person2#: I'll be taking the underground as well. Why don't we go together? I've been to the conference center only once, and I'm not sure if I can find my way around there.

Summary:
"""
inputs = tok(prompt, return_tensors='pt').to('cuda')
output = tok.decode(
    model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)
print(f'INPUT PROMPT:\n{prompt}')
print(output)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


INPUT PROMPT:

Summarize the following conversation.

Conversation:
#Person1#: Mister Ewing said we should show up at the conference center at 4:00 o'clock, right?
#Person2#: Yes, he especially asked us not to be late. Some of the people from our east york branch office are coming, and he wants to make a good impression on them. How are you getting there?
#Person1#: I was thinking of taking my car, but I think I'm just going to take the underground, because there is construction on the highway. What about you?
#Person2#: I'll be taking the underground as well. Why don't we go together? I've been to the conference center only once, and I'm not sure if I can find my way around there.

Summary:


Summarize the following conversation.

Conversation:
#Person1#: Mister Ewing said we should show up at the conference center at 4:00 o'clock, right?
#Person2#: Yes, he especially asked us not to be late. Some of the people from our east york branch office are coming, and he wants to make a good i